In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import json
from datetime import datetime
import random

# Base de dados

In [2]:
df = pd.read_excel('../Base120.xlsx')

In [3]:
def filtra_por_solucao(df, vet_solucao):
    '''Recebe um vetor solução e retorna o df filtrado para essa solução'''
    stand_prescription_map = {i + 1: vet_solucao[i] for i in range(len(vet_solucao))}
    filtered_df = df[df.apply(lambda row: row['prescrição'] == stand_prescription_map.get(row['talhao']), axis=1)]
    return filtered_df

def calcula_volume(result):
	"""
	Calculate the volume by summing specific columns of a DataFrame.
	This function takes a DataFrame `result` and returns the sum of the values
	in columns 3 to 18 (inclusive).
	Parameters:
	result (pandas.DataFrame): The input DataFrame containing the data.
	Returns:
	pandas.Series: A Series containing the sum of the specified columns.
	"""
	
    
	return result.iloc[:, 3:19].sum()

def calcula_penalidade(volume_anual):
    '''retorna penalidade por volume excedente/faltante (R$500)'''
    upper = []
    lower = []
    for v_ano in volume_anual:
        if v_ano < 140000:
            lower.append(np.abs(v_ano - 140000))
        elif v_ano > 160000:
            upper.append(np.abs(v_ano - 160000))
    return np.sum(upper + lower)*500

def calcula_vpl_aux(result, volume_ano):
    '''Calcula o VPL penalizado (função auxiliar)'''
    return np.sum(result['VPL']) - calcula_penalidade(volume_ano)

def plota_volume(df, vet_sol):
    '''Plota o gráfico do volume nos anos'''
    volume_ano = filtra_por_solucao(df, vet_sol).iloc[:, 3:19].sum()
    (volume_ano/1000).plot(kind='bar')
    plt.hlines(140, xmin=-1, xmax=16, color='red', alpha=.6)
    plt.hlines(160, xmin=-1, xmax=16, color='red', alpha=.6)
    plt.title("Volume de madeira por ano")
    plt.xlabel("Ano de Produção")
    plt.ylabel("Volume (1000 m$^3$)")

def calcula_volume_aux(df, vet_sol):
	"""
	Calculate the auxiliary volume based on the given solution vector.
	This function filters the DataFrame `df` using the solution vector `vet_sol`
	and then sums the values in columns 3 to 18 (inclusive) of the filtered DataFrame.
	Parameters:
	df (pandas.DataFrame): The input DataFrame containing the data.
	vet_sol (list or array-like): The solution vector used to filter the DataFrame.
	Returns:
	pandas.Series: A Series containing the sum of the values in columns 3 to 18 of the filtered DataFrame.
	"""
    
	return filtra_por_solucao(df, vet_sol).iloc[:, 3:19].sum()

def calcula_vpl_total(df, vet_solucao):
    '''Calcula o VPL total penalizado'''
    result = filtra_por_solucao(df, vet_solucao)
    volume_ano = calcula_volume(result)
    return calcula_vpl_aux(result, volume_ano)

def encontra_pior_ano(df, sol):
    volume_ano = calcula_volume_aux(df, sol)
    return np.argmax(np.abs(volume_ano-160000)) + 1

def numpy_to_python(obj):
    '''Converte np.int e np.float para int e float dos dados do dicionário'''
    if isinstance(obj, (np.integer, np.floating)):
        return obj.item()
    
def python_to_numpy(dicionario):
    '''Converte int e float para np.int e np.float dos dados do dicionário'''
    for chave, subdict in dicionario.items():
        for subchave, valor in subdict.items():
            if isinstance(valor, int):
                subdict[subchave] = np.int64(valor)
            elif isinstance(valor, float):
                subdict[subchave] = np.float64(valor)
                
    dicionario = {np.int64(key): valor for key, valor in dicionario.items()}
    return dicionario

---

## Alteração na base de dados

In [4]:
df_2 = df.drop_duplicates(subset=['talhao'] + df.columns[3:].tolist(), keep='first')
df_2.to_csv('../base_reduzida.csv', index=False)

---

# Algoritmo Genético

## Soluções importadas


In [14]:
with open("../solucoes_alpha.json", "r") as arquivo: # soluções geradas por heurística construtiva
    solucoes_alpha = json.load(arquivo)
solucoes_alpha = [np.array(sol) for sol in solucoes_alpha]

In [19]:
sol = solucoes_alpha[0]
sol

array([37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 40, 64, 40, 40, 64,
       64, 40, 40, 40, 64, 64, 40, 40, 40, 40, 64, 40, 40, 64, 40, 64, 40,
       43, 67, 67, 67, 67, 43, 67, 67, 67, 67, 43, 67, 43, 43, 43, 67, 67,
       67, 67, 67, 67, 43, 67, 43, 67, 67, 43, 67, 43, 43, 43, 43, 52, 52,
       52, 52, 52, 52, 52, 52, 76, 76, 52, 52, 52, 76, 76, 52, 79, 79, 79,
       79, 79, 79, 79, 79, 79, 28, 79, 79, 79, 79, 79, 79, 79, 79, 28, 79,
       79, 79, 79, 79, 10, 10, 10, 10, 10, 10, 79, 10, 10, 10, 10, 10, 10,
       10])

In [24]:
np.random.shuffle(sol[1,2,3])

IndexError: too many indices for array: array is 1-dimensional, but 3 were indexed

In [28]:
sol(1)

TypeError: 'numpy.ndarray' object is not callable

In [16]:
def gera_populacao_inicial(n_pop, n_talhoes):
    '''Gera uma população inicial de soluções'''
    populacao = np.random.randint(1, 121, size=(round(n_pop*0.8), n_talhoes))
    melhor_sol = solucoes_alpha[2]
    for i in range(n_pop-round(n_pop*0.8)):
        random.shuffle(melhor_sol)
    return populacao
gera_populacao_inicial(20, 120)

array([[ 46,  30,  88, 106,  99,  62,  66, 106, 101,  79,  39,  14,  33,
         12,  67,   4,  87,  94,  90,   4, 107, 100, 114,  47,  12, 112,
         30,   2,  35,  48,  77,  95, 114,  62,  55,   5,  22,  20,  13,
         68,  59,  68,  12,  33,  24,  90, 120,  21,  22,  24, 119,  89,
        117,  14,  98, 107,  73,  42,   8,  58,   3,  70,  41,  17,  26,
         90,  24, 112,   9,  93, 112,  37,  10,  16,  12,  12,  79,  46,
         75, 107,  74,  65,  48,  87,  50,  15, 110,  65,   9,  70,  29,
        120,  93,  30,  66,  35,  76,  91,  23,  90,  59,  36,  25,  86,
        101,  24,  19,  67,  15,  16,  35,  54,  67,  42, 103,  38,  15,
         20,  81,  93],
       [ 76, 102, 112,  68,  34,  65, 109, 116, 116,  54,  32,  31,  90,
         64,  50,  28,  71,  62, 115,  97,  86,  31, 110,  85, 119,  98,
          9,  24,  29,   7, 112, 113,  74,  95, 114,   9,  68,  44,   3,
         63,  94,  59,  54,  73,  40,  86,  62,  73,  46,  77,  23,  13,
          6, 120, 104, 120,